# SAP Cloud ALM Process Authoring API Test and Demo

This notebook contains examples of SAP Business Hub API calls for SAP Cloud ALM for implementation. The specific APIs shown below are for Process Authoring.

The API information and specification is available here:

* https://api.sap.com/package/SAPCloudALM/rest - SAP Cloud ALM
* https://api.sap.com/api/CALM_PMGE/overview - SAP Cloud ALM Process Authoring

Please note the license and other terms and conditions contained in this notebook's repository: https://github.com/SAP-samples/cloud-alm-api-examples

## Python Dependencies Required

In order to run the samples in this notebook, install the following dependencies:

* Jupyter integration in Visual Studio Code: https://code.visualstudio.com/docs/python/jupyter-support 
* Python 3, a recent version, is Required. Python 3.8 was used here.
* Requests - for handling HTTP GET/POST/PATCH/DELETE Requests - https://docs.python-requests.org/en/latest/user/install/#install
* Requests-OAuthlib - for authentication with requests - https://requests-oauthlib.readthedocs.io/en/latest/index.html#installation
* Pandas - Python data analysis - https://pandas.pydata.org/docs/getting_started/install.html
* Plotly - for plotting interactive charts - https://plotly.com/python/getting-started/

## APIs called format

The PMGE public API is taking only one request format: https://<tenant url\>/api/calm-processauthoring/v1/<entity>

For example:
- Get all business processes: [GET] https://<tenant url\>/api/calm-processauthoring/v1/businessProcesses
- Get a single business process: [GET] https://<tenant url\>/api/calm-processauthoring/v1/businessProcesses/{businessProcessID}
- Create a business process: [POST] https://<tenant url\>/api/calm-processauthoring/v1/businessProcesses
- Edit a business process: [PATCH] https://<tenant url\>/api/calm-processauthoring/v1/businessProcesses/{businessProcessID} (warning: only what you provide will be changed)
- Delete a business process: [DELETE] https://<tenant url\>/api/calm-processauthoring/v1/businessProcesses/{businessProcessID}

You can follow this format for all entities available.

---
### Update 2023-01-27 - First API Publication

Process Authoring is released


---

## Authentication information

You must create a python module file called `apidata.py` and put the information specific to your tenant there. This includes

* OAuth2 client ID and secret
* Token url
* Base URL for API calls

Get client ID and secret variables from an external module: this information is senstive.

These items can be retrieved from the SAP BTP Cockpit

### Format of module apidata.py for import

```python
client_id = r'get your client ID from SAP BTP Cockpit'
client_secret = r'get your client secret from SAP BTP Cockpit'
token_url = 'your token url'
base_url = 'your base url'
service_endpoint = 'your service endpoint'
version_number = 'service version'
```


In [ ]:
import apidata as ad

client_id = ad.client_id
client_secret = ad.client_secret
token_url = ad.token_url
base_url = ad.base_url
full_url = ad.base_url + '/api/' + ad.service_endpoint + '/' + ad.version_number

### Get token for authentication

Call OAuth token API with credential information. Add the resulting header to all requests.

See Requests-OAuthlib documentation for Backend Application Flow:

* https://requests-oauthlib.readthedocs.io/en/latest/oauth2_workflow.html#backend-application-flow

In [ ]:
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient

client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url=token_url, client_id=client_id,
        client_secret=client_secret)

hed = {'Authorization': 'Bearer ' + token['access_token']}

# Common Utils (dependencies, methods, entities)

## Required dependencies

In [ ]:
import requests
import pandas as pd
import datetime
import json
from tabulate import tabulate
from enum import Enum
import copy

## Methods and enums

In [ ]:
class ResponseType(Enum):
    ALL = 1
    SINGLE = 2
    COUNT = 3

class ResponseCode(Enum):
    OK = 200
    CREATED = 201
    DELETED = 204
    ETAG_FAILURE = 409

In [ ]:
def getDateNow():
    return datetime.datetime.now().strftime("%d-%b-%Y(%H:%M:%S.%f)")

def getKey(dataframe, keyName):
    return dataframe.to_dict().get(keyName)[0]

# Parse JSON into Pandas Dataframe
# This takes the data returned from the API, which is in JSON format, and places it into a dataframe for further processing and analysis.
def handle_response(res, res_type):
    if res.json() is not None:
        if res_type == ResponseType.ALL:
            return pd.json_normalize(res.json()['value'])
        elif res_type == ResponseType.SINGLE:
            return pd.json_normalize(res.json())
        elif res_type == ResponseType.COUNT:
            return res.json()

    raise Exception

def handle_responseCode(response, shouldBeResponseCode):
    if response.status_code != shouldBeResponseCode.value:
        print("[ERROR] Response code returned: " + str(response.status_code) + " " + response.reason + ", should be: " + str(shouldBeResponseCode.value))
        print("[ERROR] Response error message: " + response.json()['error']['message'])

def handle_responseType(df, responseType):
    if responseType == ResponseType.COUNT:
        print("Elements found: " + str(df))
    else:
        print(tabulate(df, headers='keys', tablefmt='psql'))

def getRequest(entity, responseCode, responseType):
    print ( "============= GET REQUEST ============="
            "\nEntity: " + entity + "\n"
            "\nResponseCode: " + str(responseCode) + "\n"
            "\nResponseType: " + str(responseType) + "\n"
            "=======================================")

    response = requests.get(full_url + '/' + entity, headers=hed)

    handle_responseCode(response, responseCode)

    df = handle_response(response, responseType)

    handle_responseType(df, responseType)
    
    return response, df

def postRequest(entity, body, responseCode, responseType):
    print ( "============= POST REQUEST ============="
            "\nEntity: " + entity + "\n"
            "\nBody: " + json.dumps(body, indent=4, ensure_ascii=False) + "\n"
            "\nResponseCode: " + str(responseCode) + "\n"
            "\nResponseType: " + str(responseType) + "\n"
            "========================================")

    response = requests.post(full_url + '/' + entity, json = body, headers=hed)

    handle_responseCode(response, responseCode)

    df = handle_response(response, responseType)

    handle_responseType(df, responseType)

    return response, df

def specialPostRequest(entity, body, etag, responseCode, responseType):
    print ( "============= SPECIAL POST REQUEST ============="
            "\nEntity: " + entity + "\n"
            "\nBody: " + json.dumps(body, indent=4, ensure_ascii=False) + "\n"
            "\nEtag: " + etag + "\n"
            "\nResponseCode: " + str(responseCode) + "\n"
            "\nResponseType: " + str(responseType) + "\n"
            "========================================")
    
    postHeaders = hed
    postHeaders['If-Match'] = etag

    response = requests.post(full_url + '/' + entity, json = body, headers=postHeaders)

    handle_responseCode(response, responseCode)

    df = handle_response(response, responseType)

    handle_responseType(df, responseType)

    return response, df

def patchRequest(entity, body, etag, responseCode, responseType):
    print ( "============= PATCH REQUEST ============="
            "\nEntity: " + entity + "\n"
            "\nBody: " + json.dumps(body, indent=4, ensure_ascii=False) + "\n"
            "\nEtag: " + etag + "\n"
            "\nResponseCode: " + str(responseCode) + "\n"
            "\nResponseType: " + str(responseType) + "\n"
            "=========================================")

    patchHeaders = hed
    patchHeaders['If-Match'] = etag

    response = requests.patch(full_url + '/' + entity, json = body, headers=patchHeaders)

    handle_responseCode(response, responseCode)

    df = handle_response(response, responseType)

    handle_responseType(df, responseType)

    return response, df

def deleteRequest(entity, etag, responseCode):
    print ( "============= DELETE REQUEST ============="
            "\nEntity: " + entity + "\n"
            "\nEtag: " + etag + "\n"
            "\nResponseCode: " + str(responseCode) + "\n"
            "==========================================")

    deleteHeaders = hed
    deleteHeaders['If-Match'] = etag

    response = requests.delete(full_url + '/' + entity, headers=deleteHeaders)

    handle_responseCode(response, responseCode)

    return response

## Entities (Models)

In [ ]:
# Business Process spec
# id: UUID
# name: string, unique, mandatory, maxLength=500
# description: string, maxLength=5000

# Object returned by the API
BusinessProcess = {
    "name": "",
    "description": "",
    "id": ""
}

# Object sent to the API
BusinessProcessCreate = {
    "name": "",
    "description": ""
}

# Object sent to the API
BusinessProcessUpdate = {
    "name": "",
    "description": ""
}

In [ ]:
# Solution Process spec
# id: UUID
# name: string, maxLength=500
# description: string, maxLength=5000
# status: string, maxLength=255
# countries: string, maxLength=5000
# externalId: string, maxLength=40
# businessProcess: mandatory, object={
#   id: UUID,
#   uri: URI
# }

# Object returned by the API
SolutionProcess = {
    "id": "",
    "name": "",
    "description": "",
    "status": "",
    "countries": "",
    "externalId": "",
    "businessProcess": {
      "id": "",
      "uri": ""
    }
}

# Object sent to the API
SolutionProcessCreate = {
    "name": "",
    "description": "",
    "countries": "",
    "externalId": "",
    "businessProcess": {
        "id": "",
        "uri": ""
    }
}

# Object sent to the API
SolutionProcessUpdate = {
    "name": "",
    "description": "",
    "status": "",
    "countries": "",
    "externalId": "",
    "businessProcess": {
        "id": "",
        "uri": ""
    }
}

In [ ]:
# Asset spec
# id: UUID
# name: string, mandatory, maxLength=500
# countryUrl: Array=
# [
#   {
#       country: string, maxLength=2
#       url: string, mandatory field, maxLength=500
#   },
#   ...
# ]

# Object returned by the API
Asset = {
    "id": "",
    "name": "",
    "countryUrl": []
}

# Object sent to the API
AssetCreate = {
    "name": "",  # mandatory field
    "countryUrl": []
}

# Object sent to the API
AssetUpdate = {
    "name": "",  # mandatory field
    "countryUrl": []
}


In [ ]:
# Solution Value Flow spec
# id: UUID
# name: string, maxLength=500
# description: string, maxLength=5000
# svg: large string, maxLength=2MB
# solutionProcess: mandatory, object={
#   id: UUID,
#   uri: URI
# }

# Object returned by the API
SolutionValueFlow = {
    "id": "",
    "name": "",
    "description": "",
    "svg": "",
    "solutionProcess": {
      "id": "",
      "uri": ""
    }
}

# Object sent to the API
SolutionValueFlowUpdate = {
    "name": "",
    "description": "",
    "svg": "",
}

# No SolutionValueFlowCreate as it's 1:1 relationship with SP (SVF automatically generated)
# solutionProcess key:value pair is not updatable

In [ ]:
# Solution Process Flow spec
# id: UUID
# name: string, maxLength=500
# description: string, maxLength=5000
# solutionProcess: mandatory, object={
#   id: UUID,
#   uri: URI
# }

# Object returned by the API
SolutionProcessFlow = {
    "id": "",
    "name": "",
    "description": "",
    "solutionProcess": {
      "id": "",
      "uri": ""
    }
}

# Object sent to the API
SolutionProcessFlowUpdate = {
    "name": "",
    "description": "",
}

# No SolutionProcessFlowCreate as it's 1:1 relationship with SP (SPF automatically generated) for the moment, later it will become 1:n
# solutionProcess key:value pair is not updatable

In [ ]:
# Solution Process Flow Diagram spec
# id: UUID
# name: string, maxLength=500
# description: string, maxLength=5000
# svg: large string, maxLength=2MB
# solutionProcessFlow: mandatory, object={
#   id: UUID,
#   uri: URI
# }

# Object returned by the API
SolutionProcessFlowDiagram = {
    "id": "",
    "name": "",
    "description": "",
    "svg": "",
    "solutionProcessFlow": {
      "id": "",
      "uri": ""
    }
}

# Object sent to the API
SolutionProcessFlowDiagramCreate = {
    "name": "",
    "description": "",
    "svg": "",
}

# Object sent to the API
SolutionProcessFlowDiagramCreateFromBPMN = {
    "name": "",
    "bpmn": ""
}

# Object sent to the API
SolutionProcessFlowDiagramUpdate = {
    "name": "",
    "description": "",
    "svg": "",
}

# solutionProcessFlow key:value pair is not updatable

---
# Manage Business Process

## GET All Business Processes

Expected response: "200 OK"

In [ ]:
getRequest("businessProcesses", ResponseCode.OK, ResponseType.ALL)

## GET COUNT Business Processes

Expected response: "200 OK"

In [ ]:
getRequest("businessProcesses/$count", ResponseCode.OK, ResponseType.COUNT)

## POST (Create) new Business Process

Expected response: "201 Created"

In [ ]:
bpBody = BusinessProcessCreate

bpBody["name"] = "BP via Public API-" + getDateNow()
bpBody["description"] = "Business Process via Public API"

rsp, createdBp = postRequest("businessProcesses", bpBody, ResponseCode.CREATED, ResponseType.SINGLE)

## GET Single Business Process

Expected response: "200 OK"

### :warning: Concurrency accessing

:warning: This API believes on LIFO (last in, first out), meanings that when getting a SINGLE entry/object/row from the db returns an "etag" header key-pair. This ETAG must be provided to patch requests when you want to edit the row.

In [ ]:
rsp, oBpFound = getRequest("businessProcesses/" +  getKey(createdBp, "id"), ResponseCode.OK, ResponseType.SINGLE)

bpEtag = rsp.headers.get("etag") # This etag must be passed into PATCH headers: If-Match=<ETAG>

## PATCH (Edit) Business Process

Expected response: "200 OK"
If Etag doesn't match: response returned = 412 ETAG_NOT_MATCHING

In [ ]:
bpUpdatedBody = BusinessProcessUpdate

bpUpdatedBody["name"] = "UPDATED - BP via Public API-" + getDateNow()
bpUpdatedBody["description"] = "UPDATED - Business Process via Public API"

rsp, oBpChanged = patchRequest("businessProcesses/" +  getKey(createdBp, "id"), bpUpdatedBody, bpEtag, ResponseCode.OK, ResponseType.SINGLE)

bpEtag = rsp.headers.get("etag") # Of course, the etag has been changed with the edit request

## DELETE Business Process

Expected response: "204 No Content"
If Etag doesn't match: response returned = 412 PRECONDITION_FAILED

In [ ]:
deleteRequest("businessProcesses/" +  getKey(createdBp, "id"), bpEtag, ResponseCode.DELETED) # Warning, the object deleted is not returned

In [ ]:
print("Business Process Notebook finished")

---
# Manage Solution Process

## GET All Solution Processes

Expected response: "200 OK"

In [ ]:
getRequest("solutionProcesses", ResponseCode.OK, ResponseType.ALL)

## GET COUNT Solution Processes

Expected response: "200 OK"

In [ ]:
getRequest("solutionProcesses/$count", ResponseCode.OK, ResponseType.COUNT)

## POST (Create) new Solution Process

Expected response: "201 Created"

In [ ]:
# First, let's create the mandatory BusinessProcess before creating SolutionProcess
bpBody = BusinessProcessCreate
bpBody["name"] = "BP via Public API-" + getDateNow()
bpBody["description"] = "Business Process via Public API"
rsp, createdBp = postRequest("businessProcesses", bpBody, ResponseCode.CREATED, ResponseType.SINGLE)
bpEtag = rsp.headers.get("etag") # This etag must be passed into PATCH headers: If-Match=<ETAG>

# Now we can create the SolutionProcess

globalExternalID = "543dd419-291d-435b-b6e9-ceb7beedaf21"

spBody = SolutionProcessCreate
spBody["name"] = "SP via Public API-" + getDateNow()
spBody["description"] = "Solution Process via Public API"
spBody["countries"] = "FR, US, DE"
spBody["externalId"] = globalExternalID
spBody["businessProcess"] = {
    "id": getKey(createdBp, "id")
}

rsp, createdSp = postRequest("solutionProcesses", spBody, ResponseCode.CREATED, ResponseType.SINGLE)

## GET Single Solution Process

Expected response: "200 OK"

:warning: This API believe on LIFO (last in, first out), meanings that when getting a SINGLE entry/object/row from the db returns an "etag" header key-pair. This ETAG must be provided to patch requests when you want to edit the row.

In [ ]:
rsp, oSpFound = getRequest("solutionProcesses/" +  getKey(createdSp, "id"), ResponseCode.OK, ResponseType.SINGLE)

spEtag = rsp.headers.get("etag") # This etag must be passed into PATCH headers: If-Match=<ETAG>

## GET All Solution Process linked to an External ID

Expected response: "200 OK"

In [ ]:
getRequest("solutionProcesses?externalId=" + globalExternalID, ResponseCode.OK, ResponseType.ALL)

## PATCH (Edit) Solution Process

Expected response: "200 OK"
If Etag doesn't match: response returned = 412 ETAG_NOT_MATCHING

In [ ]:
spUpdatedBody = SolutionProcessUpdate

spUpdatedBody["name"] = "UPDATED - SP via Public API-" + getDateNow()
spUpdatedBody["description"] = "UPDATED - Solution Process via Public API"
spUpdatedBody["countries"] = "" # If countries variable is empty, the process-authoring-service will automatically add the country "XX" (XX = generic)

# Remove unchanged elements as it is a PATCH Request -> Only elements provided are changed, those non provided remain unchanged
del spUpdatedBody["businessProcess"] # You can also change the businessProcess link if you pass another bpID
del spUpdatedBody["status"]
del spUpdatedBody["externalId"]

rsp, oSpChanged = patchRequest("solutionProcesses/" +  getKey(createdSp, "id"), spUpdatedBody, spEtag, ResponseCode.OK, ResponseType.SINGLE)

spEtag = rsp.headers.get("etag") # Of course, the etag has been changed with the edit request

## PUBLISH Solution Process

Expected response: "200 OK"
If Etag doesn't match: response returned = 412 PRECONDITION_FAILED

In [ ]:
rsp, publishedSp = specialPostRequest("publishSolutionProcess/" + getKey(createdSp, "id"), {}, spEtag, ResponseCode.OK, ResponseType.SINGLE)

spEtag = rsp.headers.get("etag") # Of course, the etag has been changed with the publication request

## CREATE DRAFT Solution Process (always from an ACTIVE SP VERSION)

#### Warning: Create a Draft from a published SP will create another SP (different ID), but if this draft version is published it will replace the previous published SP (with the original ID) 

Expected response: "200 OK"
If Etag doesn't match: response returned = 412 PRECONDITION_FAILED

In [ ]:
rsp, createdDraftSp = specialPostRequest("createDraftSolutionProcess/" + getKey(createdSp, "id"), {}, spEtag, ResponseCode.CREATED, ResponseType.SINGLE)

spDraftEtag = rsp.headers.get("etag") # Of course, the etag has been changed with the publication request
spDraftID = getKey(createdDraftSp, "id")

In [ ]:
print("Original SP (Published): " + getKey(createdSp, "id"))
print("Draft SP: " + spDraftID)

## DELETE Solution Process

Expected response: "204 No Content"
If Etag doesn't match: response returned = 412 PRECONDITION_FAILED

In [ ]:
deleteRequest("solutionProcesses/" +  getKey(createdSp, "id"), spEtag, ResponseCode.DELETED) # Warning, the object deleted is not returned
deleteRequest("solutionProcesses/" +  spDraftID, spDraftEtag, ResponseCode.DELETED) # Warning, the object deleted is not returned
deleteRequest("businessProcesses/" +  getKey(createdBp, "id"), bpEtag, ResponseCode.DELETED) # Warning, the object deleted is not returned

In [ ]:
print("Solution Process Notebook finished")

---
# Manage Assets

Assets are always linked to a Solution Process.

Endpoint list:
- [GET] /api/v1/solutionProcesses/{spID}/assets
- [GET] /api/v1/solutionProcesses/{spID}/assets/\$count

- [POST] /api/v1/solutionProcesses/{spID}/assets
- [GET] /api/v1/assets
- [GET] /api/v1/assets/$\$count
- [GET] /api/v1/assets/{assetID}
- [PATCH] /api/v1/assets/{assetID}
- [DELETE] /api/v1/assets/{assetID}

---

## /api/v1/solutionProcesses/{spID}/assets endpoints

### Create pre-required entities

In [ ]:
# First, let's create the mandatory BusinessProcess before creating SolutionProcess and Assets
bpBody = BusinessProcessCreate
bpBody["name"] = "BP via Public API-" + getDateNow()
bpBody["description"] = "Business Process via Public API"
rsp, createdBp = postRequest("businessProcesses", bpBody, ResponseCode.CREATED, ResponseType.SINGLE)
bpEtag = rsp.headers.get("etag") # This etag must be passed into PATCH headers: If-Match=<ETAG>

# Now we can create the SolutionProcess
spBody = SolutionProcessCreate
spBody["name"] = "SP via Public API-" + getDateNow()
spBody["description"] = "Solution Process via Public API"
spBody["countries"] = "FR, US, DE"
spBody["businessProcess"] = {
    "id": getKey(createdBp, "id")
}
rsp, createdSp = postRequest("solutionProcesses", spBody, ResponseCode.CREATED, ResponseType.SINGLE)
spEtag = rsp.headers.get("etag") # This etag must be passed into PATCH headers: If-Match=<ETAG>

### GET All Assets

Expected response: "200 OK"

In [ ]:
getRequest("solutionProcesses/" + getKey(createdSp, "id") + "/assets", ResponseCode.OK, ResponseType.ALL)

### GET COUNT Assets

Expected response: "200 OK"

In [ ]:
getRequest("solutionProcesses/" + getKey(createdSp, "id") + "/assets/$count", ResponseCode.OK, ResponseType.COUNT)

### POST (Create) new Asset

Expected response: "201 Created"

In [ ]:
assetBody = AssetCreate
assetBody["name"] = "Asset via Public API-" + getDateNow()
assetBody["countryUrl"] = [
    {
        "country": "US",
        "url": "https://en.wikipedia.org/wiki/SAP"  # Random url, just for the demo
    },
    {
        "country": "FR",
        "url": "https://www.sap.com/france/index.html" # Random url, just for the demo
    }
]

rsp, createdAsset = postRequest("solutionProcesses/" + getKey(createdSp, "id") + "/assets", assetBody, ResponseCode.CREATED, ResponseType.SINGLE)

### GET All Assets again

Expected response: "200 OK"

In [ ]:
getRequest("solutionProcesses/" + getKey(createdSp, "id") + "/assets", ResponseCode.OK, ResponseType.ALL)

### GET COUNT Assets again

Expected response: "200 OK"

In [ ]:
getRequest("solutionProcesses/" + getKey(createdSp, "id") + "/assets/$count", ResponseCode.OK, ResponseType.COUNT)

---

## /api/v1/assets endpoints

### GET All Assets

Expected response: "200 OK"

In [ ]:
getRequest("assets", ResponseCode.OK, ResponseType.ALL)

### GET COUNT Assets

Expected response: "200 OK"

In [ ]:
getRequest("assets/$count", ResponseCode.OK, ResponseType.COUNT)

### GET Single Asset

Expected response: "200 OK"

:warning: This API believe on LIFO (last in, first out), meanings that when getting a SINGLE entry/object/row from the db returns an "etag" header key-pair. This ETAG must be provided to patch requests when you want to edit the row.

In [ ]:
rsp, oAssetFound = getRequest("assets/" +  getKey(createdAsset, "id"), ResponseCode.OK, ResponseType.SINGLE)

assetEtag = rsp.headers.get("etag") # This etag must be passed into PATCH headers: If-Match=<ETAG>

### PATCH (Edit) Asset

Expected response: "200 OK"
If Etag doesn't match: response returned = 412 ETAG_NOT_MATCHING

In [ ]:
assetUpdatedBody = AssetUpdate

# The countryUrl list need to be fully passed to the patch request. Otherwise, only what has been passed will be kept
assetUpdatedBody["name"] = "UPDATED - Asset via Public API-" + getDateNow()
assetUpdatedBody["countryUrl"] = [
    {
        "country": "DE",
        "url": "https://api.sap.com/package/SAPCloudALM/rest"  # Random url, just for the demo
    }
]

rsp, oAssetChanged = patchRequest("assets/" +  getKey(createdAsset, "id"), assetUpdatedBody, assetEtag, ResponseCode.OK, ResponseType.SINGLE)

assetEtag = rsp.headers.get("etag") # Of course, the etag has been changed with the edit request

### DELETE Asset

Expected response: "204 No Content"
If Etag doesn't match: response returned = 412 PRECONDITION_FAILED

In [ ]:
deleteRequest("assets/" +  getKey(createdAsset, "id"), assetEtag, ResponseCode.DELETED) # Warning, the object deleted is not returned

### Clean pre-required entities

In [ ]:
deleteRequest("solutionProcesses/" +  getKey(createdSp, "id"), spEtag, ResponseCode.DELETED) # Warning, the object deleted is not returned
deleteRequest("businessProcesses/" +  getKey(createdBp, "id"), bpEtag, ResponseCode.DELETED) # Warning, the object deleted is not returned

In [ ]:
print("Asset Notebook finished")

---
# Manage Solution Value Flow

Solution Value Flow is always linked to a Solution Process (1:1 relationship)

Endpoint list:
- [GET] /api/v1/solutionProcesses/{spID}/solutionValueFlowDiagram

- [GET] /api/v1/solutionValueFlowDiagrams
- [GET] /api/v1/solutionValueFlowDiagrams/\$count
- [GET] /api/v1/solutionValueFlowDiagrams/{solutionValueFlowDiagramID}
- [PATCH] /api/v1/solutionValueFlowDiagrams/{solutionValueFlowDiagramID}

---

## /api/v1/solutionProcesses/{spID}/solutionValueFlowDiagram endpoints

### Create pre-required entities

In [ ]:
# First, let's create the mandatory BusinessProcess before creating SolutionProcess and SolutionValueFlow
bpBody = BusinessProcessCreate
bpBody["name"] = "BP via Public API-" + getDateNow()
bpBody["description"] = "Business Process via Public API"
rsp, createdBp = postRequest("businessProcesses", bpBody, ResponseCode.CREATED, ResponseType.SINGLE)
bpEtag = rsp.headers.get("etag") # This etag must be passed into PATCH headers: If-Match=<ETAG>

# Now we can create the SolutionProcess
spBody = SolutionProcessCreate
spBody["name"] = "SP via Public API-" + getDateNow()
spBody["description"] = "Solution Process via Public API"
spBody["countries"] = "FR, US, DE"
spBody["businessProcess"] = {
    "id": getKey(createdBp, "id")
}
rsp, createdSp = postRequest("solutionProcesses", spBody, ResponseCode.CREATED, ResponseType.SINGLE)
spEtag = rsp.headers.get("etag") # This etag must be passed into PATCH headers: If-Match=<ETAG>

### GET The Single SolutionValueFlow in specific SP

Expected response: "200 OK"

In [ ]:
rsp, oSvf = getRequest("solutionProcesses/" + getKey(createdSp, "id") + "/solutionValueFlowDiagram", ResponseCode.OK, ResponseType.SINGLE)
svfID = getKey(oSvf, "id")

---

## /api/v1/solutionValueFlowDiagrams endpoints

### GET All SolutionValueFlows

Expected response: "200 OK"

In [ ]:
getRequest("solutionValueFlowDiagrams", ResponseCode.OK, ResponseType.ALL)

### GET COUNT SolutionValueFlows

Expected response: "200 OK"

In [ ]:
getRequest("solutionValueFlowDiagrams/$count", ResponseCode.OK, ResponseType.COUNT)

### GET Single SolutionValueFlow

Expected response: "200 OK"

:warning: This API believe on LIFO (last in, first out), meanings that when getting a SINGLE entry/object/row from the db returns an "etag" header key-pair. This ETAG must be provided to patch requests when you want to edit the row.

In [ ]:
rsp, oSVFFound = getRequest("solutionValueFlowDiagrams/" + svfID, ResponseCode.OK, ResponseType.SINGLE)

svfEtag = rsp.headers.get("etag") # This etag must be passed into PATCH headers: If-Match=<ETAG>

### PATCH (Edit) SolutionValueFlow

Expected response: "200 OK"
If Etag doesn't match: response returned = 412 ETAG_NOT_MATCHING

In [ ]:
svfUpdatedBody = SolutionValueFlowUpdate

svfUpdatedBody["name"] = "UPDATED - SVF via Public API-" + getDateNow()
svfUpdatedBody["svg"] = "<svg width=\"800px\" height=\"800px\" viewBox=\"00512512\" xmlns=\"http: //www.w3.org/2000/svg\" xmlns:xlink=\"http: //www.w3.org/1999/xlink\" aria-hidden=\"true\" role=\"img\" class=\"iconifyiconify--fxemoji\" preserveAspectRatio=\"xMidYMidmeet\"><path fill=\"#B0E9FF\" d=\"M441.518268.098c-5.510-10.859.682-15.981.945a69.16769.1670001.767-15.489c0-38.123-30.905-69.028-69.028-69.028a68.70468.704000-38.98312.065c-12.949-49.891-58.284-86.729-112.225-86.729c-64.0310-115.93951.908-115.939115.939c-48.1410-87.16839.026-87.16887.168c048.14139.02687.16887.16887.168h350.388c36.738066.52-29.78266.52-66.52c0-36.737-29.782-66.519-66.52-66.519z\"></path></svg>"

rsp, oSVFChanged = patchRequest("solutionValueFlowDiagrams/" + svfID, svfUpdatedBody, svfEtag, ResponseCode.OK, ResponseType.SINGLE)

svfEtag = rsp.headers.get("etag") # Of course, the etag has been changed with the edit request

### Clean pre-required entities

In [ ]:
deleteRequest("solutionProcesses/" +  getKey(createdSp, "id"), spEtag, ResponseCode.DELETED) # Warning, the object deleted is not returned
deleteRequest("businessProcesses/" +  getKey(createdBp, "id"), bpEtag, ResponseCode.DELETED) # Warning, the object deleted is not returned
# Clean SolutionValueFlow entity is automatically made when the solutionProcess is deleted.

In [ ]:
print("SolutionValueFlow Notebook finished")

---
# Manage Solution Process Flow

Solution Process Flow is always linked to a Solution Process (1:1 relationship)

Endpoint list:
- [GET] /api/v1/solutionProcesses/{spID}/solutionProcessFlows
- [GET] /api/v1/solutionProcesses/{spID}/solutionProcessFlows/\$count

- [GET] /api/v1/solutionProcessFlows
- [GET] /api/v1/solutionProcessFlows/\$count
- [GET] /api/v1/solutionProcessFlows/{solutionProcessFlowID}
- [PATCH] /api/v1/solutionProcessFlows/{solutionProcessFlowID}

---

## /api/v1/solutionProcesses/{spID}/solutionProcessFlows endpoints

### Create pre-required entities

In [ ]:
# First, let's create the mandatory BusinessProcess before creating SolutionProcess and SolutionProcessFlow
bpBody = BusinessProcessCreate
bpBody["name"] = "BP via Public API-" + getDateNow()
bpBody["description"] = "Business Process via Public API"
rsp, createdBp = postRequest("businessProcesses", bpBody, ResponseCode.CREATED, ResponseType.SINGLE)
bpEtag = rsp.headers.get("etag") # This etag must be passed into PATCH headers: If-Match=<ETAG>

# Now we can create the SolutionProcess
spBody = SolutionProcessCreate
spBody["name"] = "SP via Public API-" + getDateNow()
spBody["description"] = "Solution Process via Public API"
spBody["countries"] = "FR, US, DE"
spBody["businessProcess"] = {
    "id": getKey(createdBp, "id")
}
rsp, createdSp = postRequest("solutionProcesses", spBody, ResponseCode.CREATED, ResponseType.SINGLE)
spEtag = rsp.headers.get("etag") # This etag must be passed into PATCH headers: If-Match=<ETAG>

### GET All SolutionProcessFlows in specific SP

Expected response: "200 OK"

In [ ]:
rsp, oSpfs = getRequest("solutionProcesses/" + getKey(createdSp, "id") + "/solutionProcessFlows", ResponseCode.OK, ResponseType.ALL)

spfID = getKey(oSpfs, "id")

### GET COUNT SolutionProcessFlows in specific SP

Expected response: "200 OK"

In [ ]:
getRequest("solutionProcesses/" + getKey(createdSp, "id") + "/solutionProcessFlows/$count", ResponseCode.OK, ResponseType.COUNT)

---

## /api/v1/solutionProcessFlows endpoints

### GET All SolutionProcessFlows

Expected response: "200 OK"

In [ ]:
getRequest("solutionProcessFlows", ResponseCode.OK, ResponseType.ALL)

### GET COUNT SolutionProcessFlows

Expected response: "200 OK"

In [ ]:
getRequest("solutionProcessFlows/$count", ResponseCode.OK, ResponseType.COUNT)

### GET Single SolutionProcessFlow

Expected response: "200 OK"

:warning: This API believe on LIFO (last in, first out), meanings that when getting a SINGLE entry/object/row from the db returns an "etag" header key-pair. This ETAG must be provided to patch requests when you want to edit the row.

In [ ]:
rsp, oSPFFound = getRequest("solutionProcessFlows/" + spfID, ResponseCode.OK, ResponseType.SINGLE)

spfEtag = rsp.headers.get("etag") # This etag must be passed into PATCH headers: If-Match=<ETAG>

### PATCH (Edit) SolutionProcessFlow

Expected response: "200 OK"
If Etag doesn't match: response returned = 412 ETAG_NOT_MATCHING

In [ ]:
spfUpdatedBody = SolutionProcessFlowUpdate

spfUpdatedBody["name"] = "UPDATED - SPF via Public API-" + getDateNow()
spfUpdatedBody["description"] = "Solution Process Flow via Public API"

rsp, oSPFChanged = patchRequest("solutionProcessFlows/" + spfID, spfUpdatedBody, spfEtag, ResponseCode.OK, ResponseType.SINGLE)

spfEtag = rsp.headers.get("etag") # Of course, the etag has been changed with the edit request

### Clean pre-required entities

In [ ]:
deleteRequest("solutionProcesses/" +  getKey(createdSp, "id"), spEtag, ResponseCode.DELETED) # Warning, the object deleted is not returned
deleteRequest("businessProcesses/" +  getKey(createdBp, "id"), bpEtag, ResponseCode.DELETED) # Warning, the object deleted is not returned
# Clean SolutionValueFlow entity is automatically made when the solutionProcess is deleted.

In [ ]:
print("SolutionProcessFlow Notebook finished")

---
# Manage Solution Process Flow Diagrams

Solution Process Flow Diagrams are always linked to a Solution Process Flow (1:n relationship)

Endpoint list:
- [GET] /api/v1/solutionProcessFlows/{spfID}/solutionProcessFlowDiagrams
- [GET] /api/v1/solutionProcessFlows/{spfID}/solutionProcessFlowDiagrams/\$count
- [POST] /api/v1/solutionProcessFlows/{spfID}/solutionProcessFlowDiagrams
- [POST] /api/v1/solutionProcessFlows/{spfID}/solutionProcessFlowDiagrams/bpmn
---------------

- [GET] /api/v1/solutionProcessFlowDiagrams
- [GET] /api/v1/solutionProcessFlowDiagrams/\$count
- [GET] /api/v1/solutionProcessFlowDiagrams/{spdfID}
- [PATCH] /api/v1/solutionProcessFlowDiagrams/{spdfID}
- [DELETE] /api/v1/solutionProcessFlowDiagrams/{spdfID}

---

## /api/v1/solutionProcessFlows/{spfID}/solutionProcessFlowDiagrams endpoints

### Create pre-required entities

In [ ]:
# First, let's create the mandatory BusinessProcess before creating SolutionProcess and SolutionProcessFlow and the default SolutionProcessFlowDiagram
bpBody = BusinessProcessCreate
bpBody["name"] = "BP via Public API-" + getDateNow()
bpBody["description"] = "Business Process via Public API"
rsp, createdBp = postRequest("businessProcesses", bpBody, ResponseCode.CREATED, ResponseType.SINGLE)
bpEtag = rsp.headers.get("etag") # This etag must be passed into PATCH headers: If-Match=<ETAG>

# Now we can create the SolutionProcess
spBody = SolutionProcessCreate
spBody["name"] = "SP via Public API-" + getDateNow()
spBody["description"] = "Solution Process via Public API"
spBody["countries"] = "FR, US, DE"
spBody["businessProcess"] = {
    "id": getKey(createdBp, "id")
}
rsp, createdSp = postRequest("solutionProcesses", spBody, ResponseCode.CREATED, ResponseType.SINGLE)
spEtag = rsp.headers.get("etag") # This etag must be passed into PATCH headers: If-Match=<ETAG>

# Retrieve SPF from SP
rsp, oSpfs = getRequest("solutionProcesses/" + getKey(createdSp, "id") + "/solutionProcessFlows", ResponseCode.OK, ResponseType.ALL)
spfID = getKey(oSpfs, "id")

### GET All SolutionProcessFlowDiagrams in specific SPF

Expected response: "200 OK"

In [ ]:
rsp, oSpfds = getRequest("solutionProcessFlows/" + spfID + "/solutionProcessFlowDiagrams", ResponseCode.OK, ResponseType.ALL)

### GET COUNT SolutionProcessFlowDiagrams in specific SPF

Expected response: "200 OK"

In [ ]:
getRequest("solutionProcessFlows/" + spfID + "/solutionProcessFlowDiagrams/$count", ResponseCode.OK, ResponseType.COUNT)

### POST (Create) new Solution Process Flow Diagram

Expected response: "201 Created"

In [ ]:
spfdBody = SolutionProcessFlowDiagramCreate
spfdBody["name"] = "SPFD via Public API - " + getDateNow()
spfdBody["description"] = "Solution Process Flow Diagram via Public API"
spfdBody["svg"] = "<svg width=\"240\" height=\"320\" viewBox=\"00240320\" fill=\"none\" xmlns=\"http: //www.w3.org/2000/svg\"><path d=\"M228319.5H12C5.64873319.50.5314.3510.5308V12C0.55.648735.648730.5120.5H159.175C159.5710.5159.9520.656748160.2330.936021L239.05779.2681C239.34179.5497239.579.9326239.580.3321V308C239.5314.351234.351319.5228319.5Z\" fill=\"#FFF9F2\" stroke=\"#FFC681\"/><path d=\"M160.51.20711L238.79379.5H172C165.64979.5160.574.3513160.568V1.20711Z\" fill=\"#FFF9F2\" stroke=\"#FFC681\"/><path d=\"M68.8438244.072C69.1738250.69974.7344254.91483.1387254.914C92.0254254.91497.5352250.49697.5352243.387C97.5352237.82694.4121234.72986.9727233.027L82.7578232.062C78.2129230.99676.3594229.54976.3594227.035C76.3594223.86179.1523221.77983.3672221.779C87.3789221.77990.2227223.83690.6797227.111H96.9512C96.6465220.8491.0859216.44783.3926216.447C75.1914216.44769.7578220.86569.7578227.467C69.7578232.972.8809236.17679.5332237.699L84.2812238.791C88.9785239.88390.9844241.50890.9844244.174C90.9844247.29787.8613249.55783.5449249.557C78.8984249.55775.6484247.42475.2422244.072H68.8438Z\" fill=\"#FF8C02\"/><path d=\"M121.59254L134.26217.361H127.278L118.137246.51H117.705L108.489217.361H101.252L114.049254H121.59Z\" fill=\"#FF8C02\"/><path d=\"M170.731239.375V234.805H155.522V239.807H164.333L164.307240.568C164.206245.748160.397249.227154.811249.227C148.286249.227144.198244.047144.198235.617C144.198227.314148.21222.135154.608222.135C159.305222.135162.581224.471163.901228.711H170.426C169.258221.246163.063216.447154.608216.447C144.173216.447137.495223.938137.495235.668C137.495247.525144.096254.914154.71254.914C164.485254.914170.731248.846170.731239.375Z\" fill=\"#FF8C02\"/><path fill-rule=\"evenodd\" clip-rule=\"evenodd\" d=\"M99117C96.790911795118.79195121V148.939C94.3386148.97993.671714993149C75.326914961134.67361117C6199.326975.3269859385C110.6738512599.3269125117H99ZM125117H155C157.209117159118.791159121V177C159179.209157.209181155181H99C96.790918195179.20995177V148.939C111.741147.906125134.001125117Z\" fill=\"#FFC681\"/></svg>"

rsp, createdSpfd = postRequest("solutionProcessFlows/" + spfID + "/solutionProcessFlowDiagrams", spfdBody, ResponseCode.CREATED, ResponseType.SINGLE)

### GET All SolutionProcessFlowDiagrams in specific SPF again to see the post creation

Expected response: "200 OK"

In [ ]:
rsp, oSpfds = getRequest("solutionProcessFlows/" + spfID + "/solutionProcessFlowDiagrams", ResponseCode.OK, ResponseType.ALL)

### GET COUNT SolutionProcessFlowDiagrams in specific SPF again to see the post creation

Expected response: "200 OK"

In [ ]:
getRequest("solutionProcessFlows/" + spfID + "/solutionProcessFlowDiagrams/$count", ResponseCode.OK, ResponseType.COUNT)

### POST Create SolutionProcessFlowDiagram from BPMN

In [ ]:
spfdBPMNBody = SolutionProcessFlowDiagramCreateFromBPMN
spfdBPMNBody["name"] = "SPFD from BPMN via Public API - " + getDateNow()
spfdBPMNBody["bpmn"] = "<?xml version='1.0' encoding='UTF-8'?><bpmn2:definitions id='bde14dae-c0cf-4134-9bb2-1117d9e2710c' xmlns:xsi='http://www.w3.org/2001/XMLSchema-instance' xmlns:bpmn2='http://www.omg.org/spec/BPMN/20100524/MODEL' xmlns:bpmndi='http://www.omg.org/spec/BPMN/20100524/DI' xmlns:di='http://www.omg.org/spec/DD/20100524/DI' xmlns:dc='http://www.omg.org/spec/DD/20100524/DC' targetNamespace='http://www.sap.com/bpmn2/'><bpmn2:collaboration id='collaboration-bde14dae-c0cf-4134-9bb2-1117d9e2710c'><bpmn2:participant id='a60dc9b79-757c-4923-ac64-783d1e974ed6' name='Pool' processRef='process-a60dc9b79-757c-4923-ac64-783d1e974ed6' /><bpmn2:participant id='a93d2d449-606c-4244-ac06-f4a273aaefdd' name='Pool' processRef='process-a93d2d449-606c-4244-ac06-f4a273aaefdd' /></bpmn2:collaboration><bpmn2:process id='process-a60dc9b79-757c-4923-ac64-783d1e974ed6'><bpmn2:laneSet id='laneset-a60dc9b79-757c-4923-ac64-783d1e974ed6'><bpmn2:lane id='a0a6b8882-27b2-41e5-a737-216779d14794' name='Lane'><bpmn2:flowNodeRef>a9e53989f-3a8e-40d1-895d-c6aefeba512b</bpmn2:flowNodeRef><bpmn2:flowNodeRef>a9cf2acce-80ef-4cf1-8674-51fb0ba037fc</bpmn2:flowNodeRef></bpmn2:lane></bpmn2:laneSet><bpmn2:startEvent id='a9e53989f-3a8e-40d1-895d-c6aefeba512b' name='X+_Y+' isInterrupting='true' /><bpmn2:endEvent id='a9cf2acce-80ef-4cf1-8674-51fb0ba037fc' name='X+_Y-' /></bpmn2:process><bpmn2:process id='process-a93d2d449-606c-4244-ac06-f4a273aaefdd'><bpmn2:laneSet id='laneset-a93d2d449-606c-4244-ac06-f4a273aaefdd'><bpmn2:lane id='c28aae43-48ce-4c87-be38-32ce4afdc4a7' name='Lane'><bpmn2:flowNodeRef>a5e95b60-94f3-4fbc-9cc8-c35b7c448ecb</bpmn2:flowNodeRef><bpmn2:flowNodeRef>a61c784fe-ed34-47a2-9615-23dfaaa48425</bpmn2:flowNodeRef></bpmn2:lane></bpmn2:laneSet><bpmn2:startEvent id='a5e95b60-94f3-4fbc-9cc8-c35b7c448ecb' name='X-_Y-' isInterrupting='true' /><bpmn2:endEvent id='a61c784fe-ed34-47a2-9615-23dfaaa48425' name='X-_Y+' /></bpmn2:process><bpmn2:process id='process-bde14dae-c0cf-4134-9bb2-1117d9e2710c'><bpmn2:property id='7d41a2b4-e874-4a83-a5a5-96c5ea95faf3' name='boundaryNotVisibleProperty' /><bpmn2:exclusiveGateway id='a41a2bb88-1576-4db9-8564-42ccf2997608' name='THIS-IS_0_0' /></bpmn2:process><bpmndi:BPMNDiagram id='diagram-bde14dae-c0cf-4134-9bb2-1117d9e2710c' resolution='96.0'><bpmndi:BPMNPlane bpmnElement='collaboration-bde14dae-c0cf-4134-9bb2-1117d9e2710c'><bpmndi:BPMNShape id='symbol-a60dc9b79-757c-4923-ac64-783d1e974ed6' bpmnElement='a60dc9b79-757c-4923-ac64-783d1e974ed6' isHorizontal='false'><dc:Bounds x='164' y='-252' width='120' height='620' /><bpmndi:BPMNLabel /></bpmndi:BPMNShape><bpmndi:BPMNShape id='symbol-a0a6b8882-27b2-41e5-a737-216779d14794' bpmnElement='a0a6b8882-27b2-41e5-a737-216779d14794' isHorizontal='false'><dc:Bounds x='164' y='-222' width='120' height='590' /><bpmndi:BPMNLabel /></bpmndi:BPMNShape><bpmndi:BPMNShape id='symbol-a9e53989f-3a8e-40d1-895d-c6aefeba512b' bpmnElement='a9e53989f-3a8e-40d1-895d-c6aefeba512b'><dc:Bounds x='208' y='90' width='32' height='32' /><bpmndi:BPMNLabel /></bpmndi:BPMNShape><bpmndi:BPMNShape id='symbol-a9cf2acce-80ef-4cf1-8674-51fb0ba037fc' bpmnElement='a9cf2acce-80ef-4cf1-8674-51fb0ba037fc'><dc:Bounds x='208' y='-74' width='32' height='32' /><bpmndi:BPMNLabel /></bpmndi:BPMNShape><bpmndi:BPMNShape id='symbol-a93d2d449-606c-4244-ac06-f4a273aaefdd' bpmnElement='a93d2d449-606c-4244-ac06-f4a273aaefdd' isHorizontal='false'><dc:Bounds x='-288' y='-252' width='120' height='620' /><bpmndi:BPMNLabel /></bpmndi:BPMNShape><bpmndi:BPMNShape id='symbol-c28aae43-48ce-4c87-be38-32ce4afdc4a7' bpmnElement='c28aae43-48ce-4c87-be38-32ce4afdc4a7' isHorizontal='false'><dc:Bounds x='-288' y='-222' width='120' height='590' /><bpmndi:BPMNLabel /></bpmndi:BPMNShape><bpmndi:BPMNShape id='symbol-a5e95b60-94f3-4fbc-9cc8-c35b7c448ecb' bpmnElement='a5e95b60-94f3-4fbc-9cc8-c35b7c448ecb'><dc:Bounds x='-244' y='-101' width='32' height='32' /><bpmndi:BPMNLabel /></bpmndi:BPMNShape><bpmndi:BPMNShape id='symbol-a61c784fe-ed34-47a2-9615-23dfaaa48425' bpmnElement='a61c784fe-ed34-47a2-9615-23dfaaa48425'><dc:Bounds x='-244' y='90' width='32' height='32' /><bpmndi:BPMNLabel /></bpmndi:BPMNShape><bpmndi:BPMNShape id='symbol-a41a2bb88-1576-4db9-8564-42ccf2997608' bpmnElement='a41a2bb88-1576-4db9-8564-42ccf2997608' isMarkerVisible='false'><dc:Bounds x='-4.5' y='-4.5' width='41' height='41' /><bpmndi:BPMNLabel /></bpmndi:BPMNShape></bpmndi:BPMNPlane></bpmndi:BPMNDiagram></bpmn2:definitions>"

rsp, createdSpfdBPMN = postRequest("solutionProcessFlows/" + spfID + "/solutionProcessFlowDiagrams/bpmn", spfdBPMNBody, ResponseCode.CREATED, ResponseType.SINGLE)

---

## /api/v1/solutionProcessFlowDiagrams endpoints

### GET All SolutionProcessFlowDiagrams

Expected response: "200 OK"

In [ ]:
getRequest("solutionProcessFlowDiagrams", ResponseCode.OK, ResponseType.ALL)

### GET COUNT SolutionProcessFlowDiagrams

Expected response: "200 OK"

In [ ]:
getRequest("solutionProcessFlowDiagrams/$count", ResponseCode.OK, ResponseType.COUNT)

### GET Single SolutionProcessFlowDiagram

Expected response: "200 OK"

:warning: This API believe on LIFO (last in, first out), meanings that when getting a SINGLE entry/object/row from the db returns an "etag" header key-pair. This ETAG must be provided to patch requests when you want to edit the row.

In [ ]:
rsp, oSPFDFound = getRequest("solutionProcessFlowDiagrams/" + getKey(createdSpfd, "id"), ResponseCode.OK, ResponseType.SINGLE)

spfdEtag = rsp.headers.get("etag") # This etag must be passed into PATCH headers: If-Match=<ETAG>

### PATCH (Edit) SolutionProcessFlow

Expected response: "200 OK"
If Etag doesn't match: response returned = 412 ETAG_NOT_MATCHING

In [ ]:
spfdUpdatedBody = SolutionProcessFlowDiagramUpdate

spfdUpdatedBody["name"] = "UPDATED - SPFD via Public API-" + getDateNow()
spfdUpdatedBody["description"] = "Solution Process Flow Diagram updated via Public API"
spfdUpdatedBody["svg"] = "<svg width=\"800px\" height=\"800px\" viewBox=\"0010241024\" class=\"icon\"  version=\"1.1\" xmlns=\"http: //www.w3.org/2000/svg\"><path d=\"M132.416880.832a381.33681.488010762.6720381.33681.488010-762.6720Z\" fill=\"#B8CBCD\" /><path d=\"M178.216645.192c-38.632-55.696-61.088-121.656-61.088-192.440-199.12176.976-386.592395.28-386.592218.3040395.28187.472395.28386.592071.448-22.88137.992-62.2194.01600-15.992-94.112-337.432-94.112-312.7440-329.8492.536-329.8492.536z\" fill=\"#FFBAB0\" /><path d=\"M178.216645.192c-38.632-55.696-61.088-121.656-61.088-192.440-199.12176.976-386.592395.28-386.592218.3040395.28187.472395.28386.592071.448-22.88137.992-62.2194.01600-15.992-94.112-337.432-94.112-312.7440-329.8492.536-329.8492.536z\" fill=\"#FFBAB0\" /><path d=\"M845.472662.768a1616001-13.08-25.192c38.792-55.2859.296-119.19259.296-184.8240-197.416-177.232-370.592-379.28-370.592s-379.28173.176-379.28370.592c06520.136128.39258.24183.328a1616001-26.29618.232c-41.832-60.312-63.944-130.008-63.944-201.560-214.464192.184-402.592411.28-402.592s411.28188.128411.28402.592c072.248-22.512142.512-65.104203.208a15.96815.968001-13.1126.808z\" fill=\"#783741\" /><path d=\"M847.6659.872a1616001-16-16c0-29.944-130.776-75.216-327.88-75.216-192.3760-310.50443.808-310.50475.216a1616001-320c0-73.664177.544-107.216342.504-107.216178.840359.8836.832359.88107.216a1616001-1616z\" fill=\"#783741\" /><path d=\"M177.936643.872c0-50.376139.808-91.216326.504-91.216s343.8840.84343.8891.216l-37.784156.376c044.696-139.47280.936-305.1280.936-165.6560-289.704-36.24-289.704-80.936l-37.776-156.376z\" fill=\"#F6E89A\" /><path d=\"M505.424897.184c-145.6560-302.304-29.688-305.648-94.8l-37.392-154.752a15.69615.696001-0.448-3.76c0-73.664177.544-107.216342.504-107.216178.840359.8836.832359.88107.21601.264-0.1522.528-0.4483.76L826.48802.4c-3.82469.496-193.2894.784-321.05694.784zM194.032642.344l37.248154.152c0.3041.2320.4482.4960.4483.76023.30491.46464.936273.70464.936187.0720289.12-42.896289.12-64.9360-1.2640.152-2.5280.448-3.76l37.232-154.096c-4.176-29.928-133.88-73.736-327.776-73.736-189.288-0.008-306.69642.408-310.42473.68z\" fill=\"#783741\" /><path d=\"M505.424905.184c-153.7440-308.856-31.456-313.592-101.616l-35.848-148.36c-41.144-60.808-62.864-130.72-62.864-202.4640-218.728195.928-410.592419.28-410.592s419.28191.864419.28410.592c070.248-20.912138.888-60.544198.888l-36.72151.968c-5.71280.24-216.704101.584-328.992101.584zM197.296630.592c1.9922.63.4885.6324.2888.968l37.464155.056c0.4481.8480.6723.7360.6725.64013.7279.94456.936265.70456.936170.6320275.248-37.952281.176-57.4480-1.9040.168-3.280.616-5.128l37.04-153.312a23.98423.9840014.128-12.096c36.176-53.07255.304-114.09655.304-176.4640-193.16-173.496-362.592-371.28-362.592s-371.28169.432-371.28362.592c063.00819.424124.4856.168177.848z\" fill=\"#783741\" /><path d=\"M848.328685.936a1616001-16-16c0-29.944-130.776-75.216-327.88-75.216-192.3760-310.50443.808-310.50475.216a1616001-320c0-73.672177.544-107.216342.504-107.216178.840359.8836.832359.88107.216a1616001-1616z\" fill=\"#783741\" /><path d=\"M516.752309.408m-166.440a166.44166.44010332.880166.44166.44010-332.880Z\" fill=\"#FFFFFF\" /><path d=\"M516.744491.84c-100.5920-182.432-81.84-182.432-182.4320-100.681.84-182.44182.432-182.44100.60182.4481.84182.44182.440.008100.592-81.84182.432-182.44182.432zm0-332.864c-82.9520-150.43267.48-150.432150.44082.95267.48150.432150.432150.43282.9520150.44-67.488150.44-150.4320.008-82.96-67.48-150.44-150.44-150.44z\" fill=\"#783741\" /><path d=\"M828.328226.576c-31.49632.784-50.95277.216-50.952126.264079.65651.128147.2122.296172.08a330.664330.6640008.016-72.168c0-81.336-29.552-160.712-79.36-226.176z\" fill=\"#FFFFFF\" /><path d=\"M899.672540.928c-1.7840-3.56-0.296-5.28-0.896-79.56-27.816-133.016-103.04-133.016-187.1840-51.37619.68-100.15255.408-137.352a15.86415.86400112.456-4.888c4.6720.2648.9842.5611.8166.28853.28870.02482.632153.79282.632235.864025.256-2.83250.712-8.475.664a1616001-15.61612.504zm-72.432-288.52a165.208165.208000-33.864100.44c064.2837.216122.35294.288149.8242.672-16.5764.024-33.2964.024-49.9120-68.944-22.76-139.384-64.448-200.352z\" fill=\"#783741\" /><path d=\"M125.736527.648c75.384-22.416130.392-92.144130.392-174.8080-51.512-21.44-97.96-55.784-131.136-52.09666.384-83.216147.664-83.216231.04025.704350.7448.60874.904z\" fill=\"#FFFFFF\" /><path d=\"M125.736543.648a1616001-15.584-12.384347.448347.448001-9.024-78.512c0-84.16830.768-169.73686.632-240.922.8-3.566.968-5.77611.488-6.08a15.72815.72800112.2164.456c39.1237.79260.66488.44860.664142.64087.12-58.32165.312-141.824190.144-1.5040.44-3.040.656-4.5680.656zm75.968-297.024c-44.33662.368-68.576134.896-68.576206.136017.9121.55235.8884.62453.6461.056-25.576102.376-86.4102.376-153.5520-39.312-13.528-76.408-38.424-106.224zM436.408754.672a2424011-480v-96a2424011480v96zM628.408754.672a2424011-480v-96a2424011480v96z\" fill=\"#783741\" /></svg>"

rsp, oSPFDChanged = patchRequest("solutionProcessFlowDiagrams/" + getKey(createdSpfd, "id"), spfdUpdatedBody, spfdEtag, ResponseCode.OK, ResponseType.SINGLE)

spfdEtag = rsp.headers.get("etag") # Of course, the etag has been changed with the edit request

### DELETE Solution Process

Expected response: "204 No Content"
If Etag doesn't match: response returned = 412 PRECONDITION_FAILED

In [ ]:
deleteRequest("solutionProcessFlowDiagrams/" +  getKey(createdSpfd, "id"), spfdEtag, ResponseCode.DELETED) # Warning, the object deleted is not returned

### Clean pre-required entities

In [ ]:
deleteRequest("solutionProcesses/" +  getKey(createdSp, "id"), spEtag, ResponseCode.DELETED) # Warning, the object deleted is not returned
deleteRequest("businessProcesses/" +  getKey(createdBp, "id"), bpEtag, ResponseCode.DELETED) # Warning, the object deleted is not returned
# Clean SolutionValueFlow entity is automatically made when the solutionProcess is deleted.

In [ ]:
print("SolutionProcessFlowDiagram Notebook finished")

# Process Authoring DEMO 

In [ ]:
print("ALL NOTEBOOK EXECUTED. . .")